## Ai Travel Agent [Made By *Ahmed Mujtaba*]

In [3]:
# import plotly.express as px
# fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
# fig.write_html('first_figure.html', auto_open=True)

In [1]:
%pip install openai plotly mapbox

   ---------------------------------------- 0.0/139.3 kB ? eta -:--:--
   ----------- ---------------------------- 41.0/139.3 kB 2.0 MB/s eta 0:00:01
   ----------------- ---------------------- 61.4/139.3 kB 1.1 MB/s eta 0:00:01
   ------------------------------ ------- 112.6/139.3 kB 819.2 kB/s eta 0:00:01
   -------------------------------------- 139.3/139.3 kB 827.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.9 MB 2.3 MB/s eta 0:00:06
   ---------------------------------------- 0.1/11.9 MB 2.2 MB/s eta 0:00:06
    --------------------------------------- 0.2/11.9 MB 1.5 MB/s eta 0:00:08
    --------------------------------------- 0.2/11.9 MB 1.5 MB/s eta 0:00:08
    --------------------------------------- 0.3/11.9 MB 1.2 MB/s eta 0:00:10
   - -------------------------------------- 0.3/11.9 MB 1.3 MB/s eta 0:00:10
   - -------------------------------------- 0.4/11.9 MB 1.2 MB/s eta 0:00:10
  

In [1]:
import openai, os
from openai import OpenAI
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv, find_dotenv

mapbox_token = os.environ['MAPBOX_ACCESS_TOKEN']

In [3]:

# Create a base map using MapBox
mapbox_access_token = mapbox_token
px.set_mapbox_access_token(token=mapbox_token)
fig = px.scatter_mapbox(
    lat=[0], lon=[0], zoom=1,  
    mapbox_style="mapbox://styles/mapbox/streets-v11",
    # accesstoken=mapbox_access_token
)
fig.show()


In [2]:
from openai.types.beta import Assistant
from openai.types.beta.thread import Thread
from openai.types.beta.threads.thread_message import ThreadMessage
from openai.types.beta.threads.run import Run
from typing import Any
import time

class MessageItem:
    def __init__(self, role: str, content: str | Any):
        self.role: str = role
        self.content: str | Any = content   


class OpenAIBot:
    def __init__(self, name:str, instructions:str, model:str = "gpt-3.5-turbo-1106")->None:
        self.name: str = name
        self.instructions: str = instructions
        self.model: str = model
        load_dotenv(find_dotenv()) 
        self.client : OpenAI = OpenAI()
        self.assistant: Assistant = self.client.beta.assistants.create(
            name=self.name,
            instructions= self.instructions,
            tools=[{"type": "code_interpreter"}],
            model=self.model
        )
        self.thread: Thread  = self.client.beta.threads.create()
        self.messages: list[MessageItem] = []
    
    def send_message(self, message: str):
        latest_message: ThreadMessage = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=message
        )

        self.latest_run: Run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
            instructions=self.instructions
        )

        # print("message sent on thread id: ", self.thread.id)

        self.addMessage(MessageItem(role="user", content=message))

    def isCompleted(self)->bool:
        print("Status: ", self.latest_run.status)
        while self.latest_run.status != "completed":
            print("Going to sleep")
            time.sleep(1)
            self.latest_run : Run = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=self.latest_run.id
            )
            print("Latest Status: ", self.latest_run.status)
            # print("Latest Run: ", self.latest_run)
        return True
    
    def get_lastest_response(self)-> MessageItem:
        messages = self.client.beta.threads.messages.list(
            thread_id=self.thread.id
        )
        print("Response: ", messages.data[0])
        m = MessageItem(messages.data[0].role, messages.data[0].content[0].text.value)
        self.addMessage(m)
        return m

    def getMessages(self)->list[MessageItem]:
        return self.messages

    def addMessage(self, message: MessageItem)->None: 
        self.messages.append(message)

bot = OpenAIBot("Travel AI Assitant", 
        instructions="You are a personal travel ai assistant. Answer all about the notable landforms in the place user ask")
user_input = input('Where do you want to travel? : ')
prompt = f'Tell me about {user_input} and its notable landforms'
bot.send_message(prompt)
response: MessageItem = bot.get_lastest_response()
print(response)

Response:  ThreadMessage(id='msg_m0ddJE2AfQt35oVCQ3T7ehCz', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Tell me about Washington and its notable landforms'), type='text')], created_at=1704174353, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_3Ll32MiERfMNJ0qr9xVOjdHE')
